# PANDA EfficientNet-B3 Baseline with 36 x tiles_256


推論用のkernelは、https://www.kaggle.com/haqishen/panda-inference-w-36-tiles-256　を参考に作成する.


## TL/DL

* https://www.kaggle.com/iafoss/panda-16x128x128-tiles　を参考にtiling methodを利用
    * Simply setting the `N = 36` and `sz=256` then extract from median resolution
    * big imageのサイズ1単位をタイルとして（6, 6)->(6×256, 6×256)、各タイルのサイズは(256, 256)
* Create 6x6 big image from 36 tiles
* Efficientnet-B3を利用
* 損失関数はBCE loss（多クラス多分類によく使われる）
* BCE lossを使っているためラベル付けは以下のようにマルチクラスのように扱った.
    * E.g.
        * `label = [0,0,0,0,0]` means `isup_grade = 0`
        * `label = [1,1,1,0,0]` means `isup_grade = 3`
        * `label = [1,1,1,1,1]` means `isup_grade = 5`
* Augmentationはタイルごと、合成した画像ごとどちらにも行う
* 1epochごとにCosineAnnealingLRをかける?

## MEMO

学習が全て終わるまで10時間以上かかる

## Update


# FLOW

1. 各種設定(データセットのパス通したりモジュールインポートしたり)
2. ラベルの処理(StratifiedKFold)
3. モデルの構築
4. タイル化用の関数を定義
5. Datasetの構築
6. Augumentation
7. 損失関数の設定, train, validation用の関数の定義
9. Dataloaderの構築
8. KFold用の関数の定義
9. train


# **Let's coding!**

In [ ]:
# DEBUG = False ですべてのデータで学習する

DEBUG = True

## pytorch-gradual-warmup-lr

巨大なbatch sizeを扱うときにバッチサイズに比例させて学習率をあげると精度の悪化が見られないよと言う論文<br>
Accurate, Large Minibatch SGD: Training ImageNet in 1 Hour『https://arxiv.org/abs/1706.02677』 (参考：https://www.slideshare.net/iwiwi/nips17-86470238)
を元に作られたモジュール

In [ ]:
!pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git

優しい誰かが作ってくれたPyTorch用Efficient netのpathを通す<br>
元記事はhttps://www.kaggle.com/hmendonca/efficientnet-pytorch <br>
元々記事https://github.com/lukemelas/EfficientNet-PyTorch

In [ ]:
import os
import sys
sys.path = [
    '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
] + sys.path

In [ ]:
import time
import skimage.io
import numpy as np
import pandas as pd
import cv2
import PIL.Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from warmup_scheduler import GradualWarmupScheduler
from efficientnet_pytorch import model as enet
import albumentations
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
from tqdm import tqdm_notebook as tqdm

# Config

In [ ]:
data_dir = '../input/prostate-cancer-grade-assessment'
df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
image_folder = os.path.join(data_dir, 'train_images')

kernel_type = 'how_to_train_effnet_b0_to_get_LB_0.86'

enet_type = 'efficientnet-b0'
fold = 0
tile_size = 256
image_size = 256
n_tiles = 36
batch_size = 2
num_workers = 4
out_dim = 5
init_lr = 3e-4
warmup_factor = 10

warmup_epo = 1
# DEBUG = Trueのときは1epochだけ実験
n_epochs = 1 if DEBUG else 30
df_train = df_train.sample(100).reset_index(drop=True) if DEBUG else df_train

# 今回はGPUを使う
device = torch.device('cuda')

print(image_folder)

# Create Folds

ラベルをKFoldで処理する.

過学習防止、未知のデータへの汎用性を損なわないように**StratifiedKFold**(層別KFold).<br>
**StratifiedKFold**はデータのラベルの分布を維持したままKFoldしてくれるやつ.

In [ ]:
skf = StratifiedKFold(5, shuffle=True, random_state=42)
# 'fold'カラムを作成しとりあえず−1を入れておく
df_train['fold'] = -1
# StratifiedKFoldによるラベルづけ（[4:1]で[train:test]）
for i, (train_idx, valid_idx) in enumerate(skf.split(df_train, df_train['isup_grade'])):
    df_train.loc[valid_idx, 'fold'] = i
df_train.head()

# Model

モデルを構築する.<br>
今回は、Efficient Net b3の学習済みモデルを利用する.

In [ ]:
# 今回はeffiicentnet-b3を使う

pretrained_model = {
    'efficientnet-b3': '../input/efficientnet-pytorch/efficientnet-b3-c8376fa2.pth'
}

enet -> Identity() -> myfc

In [ ]:
# モデルの構築

class enetv2(nn.Module):
    def __init__(self, backbone, out_dim):
        '''
        params:
            backbone : Efficien Netの種類(今回はb3)
            out_dim : 最終層のノード数(クラス数、今回は6)
        '''
        super(enetv2, self).__init__()
        # 学習済みモデルのロード(Document参照)
        self.enet = enet.EfficientNet.from_name(backbone)
        self.enet.load_state_dict(torch.load(pretrained_model[backbone]))
        # enet最終層の恒等関数
        self.enet._fc = nn.Identity()
        # 付け加える全結合層
        # .enet._fc.in_featuresがenetの出力次元数
        self.myfc = nn.Linear(self.enet._fc.in_features, out_dim)
        

    def extract(self, x):
        return self.enet(x)

    def forward(self, x):
        x = self.extract(x)
        x = self.myfc(x)
        return x

# Dataset

In [ ]:
def get_tiles(img, mode=0):
        result = []
        h, w, c = img.shape
        pad_h = (tile_size - h % tile_size) % tile_size + ((tile_size * mode) // 2)
        pad_w = (tile_size - w % tile_size) % tile_size + ((tile_size * mode) // 2)

        img2 = np.pad(img,[[pad_h // 2, pad_h - pad_h // 2], [pad_w // 2,pad_w - pad_w//2], [0,0]], constant_values=255)
        img3 = img2.reshape(
            img2.shape[0] // tile_size,
            tile_size,
            img2.shape[1] // tile_size,
            tile_size,
            3
        )

        img3 = img3.transpose(0,2,1,3,4).reshape(-1, tile_size, tile_size,3)
        n_tiles_with_info = (img3.reshape(img3.shape[0],-1).sum(1) < tile_size ** 2 * 3 * 255).sum()
        if len(img3) < n_tiles:
            img3 = np.pad(img3,[[0,n_tiles-len(img3)],[0,0],[0,0],[0,0]], constant_values=255)
        idxs = np.argsort(img3.reshape(img3.shape[0],-1).sum(-1))[:n_tiles]
        img3 = img3[idxs]
        for i in range(len(img3)):
            result.append({'img':img3[i], 'idx':i})
        return result, n_tiles_with_info >= n_tiles


## get_tiles(img, mode=0) 関数について

今関数は入力した元データをタイル化して新たな画像にするための前処理関数

引数は、

* img : 与えられた元データ（訓練データ）1枚(tiff画像がサイズごとに3枚入っているが2枚目の中解像度のものを想定している)
* mode : デフォルトは0

返り値は、

* result :　numpy配列、


### 関数の動きを見るためのデバッグ


```python

import numpy as np
import skimage.io

n_tiles = 36
tile_size = 256
mode = 0

img = skimage.io.MultiImage('../input/prostate-cancer-grade-assessment/train_images/0005f7aaab2800f6170c399693a96917.tiff')[1]
print('input img : ', img.shape) # (7360, 6912, 3)
result = []
# h:高さ、w:幅、c:チャネル数(RGBなので３)
h, w, c = img.shape
# pad_h,wはどのくらいパディング用変数
pad_h = (tile_size - h % tile_size) % tile_size + ((tile_size * mode) // 2)
pad_w = (tile_size - w % tile_size) % tile_size + ((tile_size * mode) // 2)

print('padding : ', pad_h, pad_w) # 96 192

# padding後の画像img2

img2 = np.pad(img,[[pad_h // 2, pad_h - pad_h // 2], [pad_w // 2,pad_w - pad_w//2], [0,0]], constant_values=255)
print('img2 shape : ', img2.shape) # (7456, 7104, 3)

img3 = img2.reshape(
    img2.shape[0] // tile_size,
    tile_size,
    img2.shape[1] // tile_size,
    tile_size,
    3
)

print('img3 shape : ', img3.shape)

# PILの形式からTensor型の形式に変換
# その後1枚の画像を(256, 256, 3)にreshape

img3 = img3.transpose(0,2,1,3,4).reshape(-1, tile_size, tile_size,3)
print('img2 reshape : ', img3.shape)


# print((img3.reshape(img3.shape[0],-1)).sum(1))
# print(tile_size ** 2 * 3 * 255)

# print((img3.reshape(img3.shape[0],-1).sum(1) < tile_size ** 2 * 3 * 255))

# 画像が存在する（真っ白ではない）タイルの枚数を数えている
n_tiles_with_info = (img3.reshape(img3.shape[0],-1).sum(1) < tile_size ** 2 * 3 * 255).sum()
print('n_tiles_with_info : ', n_tiles_with_info)

# 1枚の元画像からタイル化した枚数
print(len(img3))


if len(img3) < n_tiles:
    img3 = np.pad(img3,[[0,n_tiles-len(img3)],[0,0],[0,0],[0,0]], constant_values=255)
    
# タイル化した画像のうち白色背景が少ないものから順にソート(36枚分)
# 白に近いほどピクセルレベルが255に近づく
idxs = np.argsort(img3.reshape(img3.shape[0],-1).sum(-1))[:n_tiles]
print(idxs)
print('tail枚数 : ', len(idxs))
img3 = img3[idxs]
print(img3.shape)
# for i in range(len(img3)):
#     result.append({'img':img3[i], 'idx':i})
# return result, n_tiles_with_info >= n_tiles


```

```output
input img :  (7360, 6912, 3)
padding :  64 0
img2 shape :  (7424, 6912, 3)
img3 shape :  (29, 256, 27, 256, 3)
img2 reshape :  (783, 256, 256, 3)
n_tiles_with_info :  58
783
[358 465 412 385 250 331 625 599 439 277 167 304 492 222 194 518 572 438
 545 519 626 305 332 491 112 139 546 140 411 166 573 598 384 249 359  85]
tail枚数 :  36
(36, 256, 256, 3)
```



In [ ]:
class PANDADataset(Dataset):
    def __init__(self,
                 df,
                 image_size,
                 n_tiles=n_tiles,
                 tile_mode=0,
                 rand=False,
                 transform=None,
                ):

        self.df = df.reset_index(drop=True)
        self.image_size = image_size
        self.n_tiles = n_tiles
        self.tile_mode = tile_mode
        self.rand = rand
        self.transform = transform

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_id = row.image_id
        
        tiff_file = os.path.join(image_folder, f'{img_id}.tiff')
        image = skimage.io.MultiImage(tiff_file)[1]
        tiles, OK = get_tiles(image, self.tile_mode)

        if self.rand:
            idxes = np.random.choice(list(range(self.n_tiles)), self.n_tiles, replace=False)
        else:
            idxes = list(range(self.n_tiles))

        # n_row_tiles　今回は6×6=36画像なので6
        n_row_tiles = int(np.sqrt(self.n_tiles))
        # image_sizeは256を想定
        images = np.zeros((image_size * n_row_tiles, image_size * n_row_tiles, 3))
        # tilesの辞書からタイル画像を取ってきてbig imageとしてマッピング
        for h in range(n_row_tiles):
            for w in range(n_row_tiles):
                i = h * n_row_tiles + w
    
                if len(tiles) > idxes[i]:
                    this_img = tiles[idxes[i]]['img']
                else:
                    this_img = np.ones((self.image_size, self.image_size, 3)).astype(np.uint8) * 255
                # 画像を黒色化している？
                this_img = 255 - this_img
                
                # ここが前述の「タイルレベルでのAugumentaion」に想定する部分
                # transformはAugumentaionをするオブジェクトを想定
                if self.transform is not None:
                    this_img = self.transform(image=this_img)['image']
                h1 = h * image_size
                w1 = w * image_size
                # 各タイル画像をbig image(images)にマッピング
                images[h1:h1+image_size, w1:w1+image_size] = this_img

        if self.transform is not None:
            images = self.transform(image=images)['image']
        # numpy.float32に変換
        images = images.astype(np.float32)
        # 0~1の範囲に正規化
        images /= 255
        # (h, w, c)のPIL形式を（c, h, w）に変換しPyTorchで扱えるようにしている
        images = images.transpose(2, 0, 1)

        label = np.zeros(5).astype(np.float32)
        # BCE Lossで扱うことを想定にしている（詳細は上記）
        label[:row.isup_grade] = 1.
        # tensor型にしたimageとtensor型にしたラベルをreturn
        return torch.tensor(images), torch.tensor(label)


# Augmentations

albumentarionsを利用<br>
詳しい説明は以下参照<br>
[画像データ拡張ライブラリ ~ albumentations ~](https://qiita.com/Takayoshi_Makabe/items/79c8a5ba692aa94043f7)<br>
[公式Document](https://albumentations.readthedocs.io/en/latest/api/augmentations.html?highlight=Transpose)

In [ ]:
# train用のAugumentationは、「転置」「左右反転」「上下反転」
# pがどれくらいの確率でその操作を行うか、という引数

transforms_train = albumentations.Compose([
    albumentations.Transpose(p=0.5),
    albumentations.VerticalFlip(p=0.5),
    albumentations.HorizontalFlip(p=0.5),
])
# validationのときはAugumentationしなくていいからそのまんま
transforms_val = albumentations.Compose([])

いったん作った画像を可視化してみる

In [ ]:
dataset_show = PANDADataset(df_train, image_size, n_tiles, 0, transform=transforms_train)
from pylab import rcParams
rcParams['figure.figsize'] = 20,10
for i in range(2):
    f, axarr = plt.subplots(1,5)
    for p in range(5):
        idx = np.random.randint(0, len(dataset_show))
        img, label = dataset_show[idx]
        axarr[p].imshow(1. - img.transpose(0, 1).transpose(1,2).squeeze())
        axarr[p].set_title(str(sum(label)))


# Loss

今回はBCE Lossを使う

In [ ]:
criterion = nn.BCEWithLogitsLoss()

# Train & Val

train, validation用の関数を定義

ここで注意する点は、Dataloaderはデータセットを使い切るまでバッチサイズ分データを固めて返してくれるだけ、<br>
つまりはDataloade自体は1epoch分forループを回すだけなので学習時はエポック分さらにforループで回してあげる必要がある.

---memo---<br>
Dataloaderあたりか[これ](https://qiita.com/mathlive/items/2a512831878b8018db02)が参考になる.<br>
ProgressBarを表示させるtqdmについては[これ](https://qiita.com/pontyo4/items/76145cb10e030ad8186a)がいい感じ.<br>
検証用関数のno.grad()については[これ](https://qiita.com/a_yoshii/items/598365cf3b68955e11c5)<br>


.cpu().numpy()に関してはmodel自体の計算はGPUで行っているが正解率やlossを計算するときはcpuで行いたいので一度CPUに落としてから出ないとダメ

In [ ]:
# 訓練用関数の定義

def train_epoch(loader, optimizer):
    
    '''
    params:
        loader : データローダーを想定
        optimizer: 最適化手法を想定
    '''
    
    # modelをtrainモードに
    model.train()
    #tarin_lossが記録用配列
    train_loss = []
    bar = tqdm(loader)
    for (data, target) in bar:
        
        # data(訓練データ[画像])とtarget(訓練ラベル[tensor配列])をGPUヘ
        data, target = data.to(device), target.to(device)
        # 損失関数を定義
        loss_func = criterion
        # optimizerの勾配をリセット
        optimizer.zero_grad()
        # logitsがモデルを通した出力
        # 形式は[0.1, 0.5, 0.12, 0.18, 0.05, 0.05]などを想定
        logits = model(data)
        # lossを計算
        loss = loss_func(logits, target)
        # 勾配計算
        loss.backward()
        # パラメータ更新
        optimizer.step()
        # モデルから損失だけ取得
        loss_np = loss.detach().cpu().numpy()
        # lossを記録
        train_loss.append(loss_np)
        # smooth_lossは下の方法で平滑化したやつっぽい
        smooth_loss = sum(train_loss[-100:]) / min(len(train_loss), 100)
        # lossとsmooth_lossをprint
        bar.set_description('loss: %.5f, smth: %.5f' % (loss_np, smooth_loss))
    return train_loss



# 検証用関数の定義
def val_epoch(loader, get_output=False):
    
    # modelをevalモードに
    model.eval()
    val_loss = []
    LOGITS = 
    # 実際にLOGITからどのクラスを指定したかをPREDSに保存(下のTARGETSと同様)
    PREDS = []
    # 正解ラベル（ex:[1,0,0,0,0,0]）を合計して整数化したもの(0~5の範囲をとる)の記録用配列
    TARGETS = []

    with torch.no_grad():
        for (data, target) in tqdm(loader):
            data, target = data.to(device), target.to(device)
            logits = model(data)

            loss = criterion(logits, target)
            
            # modelからの出力であるlogitsの各要素に対してsigmoid()をかけ,
            # その総和を取り、モデルから取得、小数点を切り上げ切り下げし整数化(0~5にしている)
            # この段階ではdetach()のときGPU上で行っている
            pred = logits.sigmoid().sum(1).detach().round()
            LOGITS.append(logits)
            PREDS.append(pred)
            # 正解ラベル（ex:[1,0,0,0,0,0]）を合計して整数化したもの(0~5の範囲をとる)
            TARGETS.append(target.sum(1))

            val_loss.append(loss.detach().cpu().numpy())
        val_loss = np.mean(val_loss)
        
    # 正解率を計算
    # この段階でCPUに落とす
    LOGITS = torch.cat(LOGITS).cpu().numpy()
    PREDS = torch.cat(PREDS).cpu().numpy()
    TARGETS = torch.cat(TARGETS).cpu().numpy()
    acc = (PREDS == TARGETS).mean() * 100.
    
    # ここはよくわからん
    qwk = cohen_kappa_score(PREDS, TARGETS, weights='quadratic')
    qwk_k = cohen_kappa_score(PREDS[df_valid['data_provider'] == 'karolinska'], df_valid[df_valid['data_provider'] == 'karolinska'].isup_grade.values, weights='quadratic')
    qwk_r = cohen_kappa_score(PREDS[df_valid['data_provider'] == 'radboud'], df_valid[df_valid['data_provider'] == 'radboud'].isup_grade.values, weights='quadratic')
    print('qwk', qwk, 'qwk_k', qwk_k, 'qwk_r', qwk_r)

    if get_output:
        return LOGITS
    else:
        return val_loss, acc, qwk

    

# Create Dataloader & Model & Optimizer

##GradualWarmupSchedulerの使い方

以下を参照<br>
[](http://)

In [ ]:
# 今回はfold=0
train_idx = np.where((df_train['fold'] != fold))[0]
valid_idx = np.where((df_train['fold'] == fold))[0]

df_this  = df_train.loc[train_idx]
df_valid = df_train.loc[valid_idx]

dataset_train = PANDADataset(df_this , image_size, n_tiles, transform=transforms_train)
dataset_valid = PANDADataset(df_valid, image_size, n_tiles, transform=transforms_val)

# Samplerがtrainとvalidで違うので注意
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, sampler=RandomSampler(dataset_train), num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, sampler=SequentialSampler(dataset_valid), num_workers=num_workers)

model = enetv2(enet_type, out_dim=out_dim)
# modelをGPUにおくる
model = model.to(device)

# optimizerはAdam
# lr（学習率）はwarmup_factorを使って可変にしている(いまいちよく分かっていない)
optimizer = optim.Adam(model.parameters(), lr=init_lr/warmup_factor)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs-warmup_epo)
scheduler = GradualWarmupScheduler(optimizer, multiplier=warmup_factor, total_epoch=warmup_epo, after_scheduler=scheduler_cosine)

print(len(dataset_train), len(dataset_valid))

# Run Training

In [ ]:
qwk_max = 0.
best_file = f'{kernel_type}_best_fold{fold}.pth'
for epoch in range(1, n_epochs+1):
    print(time.ctime(), 'Epoch:', epoch)
    scheduler.step(epoch-1)

    train_loss = train_epoch(train_loader, optimizer)
    val_loss, acc, qwk = val_epoch(valid_loader)

    content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {np.mean(train_loss):.5f}, val loss: {np.mean(val_loss):.5f}, acc: {(acc):.5f}, qwk: {(qwk):.5f}'
    print(content)
    with open(f'log_{kernel_type}.txt', 'a') as appender:
        appender.write(content + '\n')

    if qwk > qwk_max:
        print('score2 ({:.6f} --> {:.6f}).  Saving model ...'.format(qwk_max, qwk))
        torch.save(model.state_dict(), best_file)
        qwk_max = qwk

torch.save(model.state_dict(), os.path.join(f'{kernel_type}_final_fold{fold}.pth'))

# My Local Train Log


```
Tue June 2 15:39:21 2020 Epoch 1, lr: 0.0000300, train loss: 0.42295, val loss: 0.29257, acc: 47.50471, qwk: 0.77941
Tue June 2 15:51:56 2020 Epoch 2, lr: 0.0003000, train loss: 0.34800, val loss: 0.48723, acc: 29.09605, qwk: 0.58493
Tue June 2 16:04:28 2020 Epoch 3, lr: 0.0003000, train loss: 0.29207, val loss: 0.27091, acc: 52.49529, qwk: 0.81714
Tue June 2 16:17:01 2020 Epoch 4, lr: 0.0002965, train loss: 0.26521, val loss: 0.26736, acc: 57.15631, qwk: 0.80364
Tue June 2 16:29:33 2020 Epoch 5, lr: 0.0002921, train loss: 0.24412, val loss: 0.24422, acc: 56.07345, qwk: 0.84068
Tue June 2 16:42:05 2020 Epoch 6, lr: 0.0002861, train loss: 0.23085, val loss: 0.25306, acc: 58.05085, qwk: 0.84429
Tue June 2 16:54:38 2020 Epoch 7, lr: 0.0002785, train loss: 0.21998, val loss: 0.21920, acc: 62.14689, qwk: 0.86278
Tue June 2 17:07:10 2020 Epoch 8, lr: 0.0002694, train loss: 0.21062, val loss: 0.23400, acc: 61.91149, qwk: 0.86170
Tue June 2 17:19:47 2020 Epoch 9, lr: 0.0002589, train loss: 0.20040, val loss: 0.27417, acc: 57.10923, qwk: 0.81771
Tue June 2 17:32:25 2020 Epoch 10, lr: 0.0002471, train loss: 0.18900, val loss: 0.26732, acc: 64.92467, qwk: 0.84131
Tue June 2 17:45:05 2020 Epoch 11, lr: 0.0002342, train loss: 0.18640, val loss: 0.21936, acc: 63.27684, qwk: 0.86580
Tue June 2 17:57:42 2020 Epoch 12, lr: 0.0002203, train loss: 0.17387, val loss: 0.22863, acc: 61.25235, qwk: 0.86871
Tue June 2 18:10:23 2020 Epoch 13, lr: 0.0002055, train loss: 0.16491, val loss: 0.23071, acc: 66.85499, qwk: 0.87892
Tue June 2 18:23:00 2020 Epoch 14, lr: 0.0001901, train loss: 0.15448, val loss: 0.24338, acc: 68.45574, qwk: 0.87342
Tue June 2 18:35:39 2020 Epoch 15, lr: 0.0001743, train loss: 0.14536, val loss: 0.22043, acc: 65.11299, qwk: 0.87169
Tue June 2 18:48:18 2020 Epoch 16, lr: 0.0001581, train loss: 0.13918, val loss: 0.22007, acc: 67.65537, qwk: 0.88284
Tue June 2 19:00:55 2020 Epoch 17, lr: 0.0001419, train loss: 0.13121, val loss: 0.24287, acc: 66.71375, qwk: 0.86357
Tue June 2 19:13:35 2020 Epoch 18, lr: 0.0001257, train loss: 0.12249, val loss: 0.21583, acc: 66.80791, qwk: 0.88478
Tue June 2 19:26:14 2020 Epoch 19, lr: 0.0001099, train loss: 0.11325, val loss: 0.21401, acc: 71.13936, qwk: 0.89178
Tue June 2 19:38:55 2020 Epoch 20, lr: 0.0000945, train loss: 0.10602, val loss: 0.21250, acc: 70.00942, qwk: 0.89256
Tue June 2 19:51:32 2020 Epoch 21, lr: 0.0000797, train loss: 0.09965, val loss: 0.21149, acc: 70.33898, qwk: 0.89590
Tue June 2 20:03:59 2020 Epoch 22, lr: 0.0000658, train loss: 0.09425, val loss: 0.22203, acc: 70.76271, qwk: 0.89493
Tue June 2 20:16:28 2020 Epoch 23, lr: 0.0000529, train loss: 0.08843, val loss: 0.22948, acc: 71.70433, qwk: 0.89304
Tue June 2 20:28:56 2020 Epoch 24, lr: 0.0000411, train loss: 0.08448, val loss: 0.21200, acc: 71.18644, qwk: 0.89947
Tue June 2 20:41:25 2020 Epoch 25, lr: 0.0000306, train loss: 0.07898, val loss: 0.21873, acc: 72.55179, qwk: 0.90021
Tue June 2 20:53:53 2020 Epoch 26, lr: 0.0000215, train loss: 0.07369, val loss: 0.21842, acc: 72.64595, qwk: 0.90240
Tue June 2 21:06:20 2020 Epoch 27, lr: 0.0000139, train loss: 0.07264, val loss: 0.21501, acc: 73.21092, qwk: 0.90450
Tue June 2 21:18:49 2020 Epoch 28, lr: 0.0000079, train loss: 0.06950, val loss: 0.21616, acc: 73.35217, qwk: 0.90264
Tue June 2 21:31:16 2020 Epoch 29, lr: 0.0000035, train loss: 0.06787, val loss: 0.21195, acc: 73.11676, qwk: 0.90434
Tue June 2 21:43:43 2020 Epoch 30, lr: 0.0000009, train loss: 0.06801, val loss: 0.21014, acc: 73.11676, qwk: 0.90468
```

# Thank you for reading!